In [1]:
import pandas as pd

df = pd.read_csv("data/train.csv")
df = pd.read_csv("data/train.csv")
df = df.dropna()
X1 = df["question1"].values
X2 = df["question2"].values
y = df["is_duplicate"].values
X= [X1, X2]

In [2]:
import numpy as np
length = [len(x.split()) for x in X[0]]
length.extend([len(x.split()) for x in X[1]])

In [34]:
maxlen = 2 * int(np.mean(length))

In [35]:
print(maxlen)

22


In [4]:
%%time
import numpy as np
import gensim
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.models.keyedvectors import KeyedVectors
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk

padding = "post"
truncating = "post"
pad_val = 0
maxlen = None

sentence_params = {"emb_mode": 0, "emb_dim" : 300,
                   "remove_stopwords": False, "remove_special": False}
vector_file_path = None

def pad_sequences(sequences, maxlen=maxlen, 
                  padding=padding, truncating=truncating, value=pad_val):
    """Pad sequences to get the same length sequences

    Args:
        sequences: list(array-like), each element has to satisfy
            shape = (length,) + element_shape
        maxlen: int, optional, if not specified, maximum length among
            sequences will be set as maxlen
        padding: str, the methods of padding: 'post' or 'pre'
        value: int or float, optional value to be used for padding
    
    Returns:
        array-like, shape = (len(sequences), maxlen,) + element_shape
    """
    dtype = np.array(sequences[0]).dtype
    if maxlen is None:
        maxlen = np.max([len(seq) for seq in sequences])
    for seq in sequences:
        if list(seq):
            shape = np.array(seq).shape
            break
    num_dim = len(shape) - 1
    value = (value, value)
    processed_sequences = []
    for seq in sequences:
        if not list(seq):
            seq = np.zeros((maxlen,) + shape[1:])
        elif len(seq) >= maxlen:
            if padding == "post":
                seq = seq[:maxlen]
            else:
                seq = seq[-maxlen:]
        else:
            if padding == "post":
                pad_conf = ((0, maxlen - len(seq)),)
            else:
                pad_conf = ((maxlen - len(seq), 0),)
            for i in range(num_dim):
                pad_conf += ((0, 0),)
            seq = np.pad(np.array(seq), pad_conf,
                         mode="constant", constant_values=value)
        processed_sequences.append(seq)
    return np.array(processed_sequences, dtype=dtype)

class BasicProcessor(object):
    """Process data for estimators."""
        
    def batch_process(self, X, y=None):
        """Make sure to have numpy data for input and target"""
        if y is None:
            return np.array(X)
        else:
            return np.array(X), np.array(y)

    def batch_process_y(self, y):
        return np.array(y)

class SentenceProcessor(BasicProcessor):
    """Processor class to embed text data into numeric vectors.

    This class embeds text data through Bag of Words, Word2Vec, and
    Doc2Vec, Word2vec and Doc2Vec are based on the following papers:

    https://arxiv.org/pdf/1301.3781.pdf
    https://arxiv.org/pdf/1405.4053.pdf

    They are implemented through python package gensim:

    https://radimrehurek.com/gensim
    """

    def __init__(self, sentences=None, embedding="w2v", emb_mode=sentence_params["emb_mode"], 
                 emb_dim=sentence_params["emb_dim"],
                 remove_stopwords=sentence_params["remove_stopwords"], 
                 remove_special=sentence_params["remove_special"],
                 vector_file_path=vector_file_path, maxlen=None):
        """Initialize a processor for text data.

        Args:
            sentences: list(str)
            embedding: str, name of methods of mebeddings; we
                currently have 'w2v', 'w2v_avg', 'w2v_index',
                'd2v', and 'bow'.
            emb_mode: int, mode of vector representation. if 2
                is chosen, emb_dim becomes double
            emb_dim: int, dimention of embedding; this is used
                all of methods except for 'bow'.
            revmoe_stopswords: bool, if True, remove stopwords
                downloaded by nltk
            remove_sords: bool, if True, remove charcters other than
                alphabets and numbers
            vector_file_path: str, path to use pretrained word2vec;
                this is used only for 'w2v', 's2v_avg', and
                'w2v_index'.
        """
        self._initialize_processor(sentences, embedding, emb_mode, emb_dim,
                                   remove_stopwords, remove_special,
                                   maxlen, vector_file_path)
        
    def batch_process(self, sentences, labels=None):
        if labels is None:
            return self.sentence2vec(sentences)
        else:
            return self.sentence2vec(sentences), np.array(labels)

    def sentence2vec(self, sentences, remove_stopwords=None,
                     remove_special=None):
        """Convert text data into numeric.

        Args:
            sentences: list(str)
            embedding: str, name of methods of mebeddings; we
                currently have 'w2v', 'w2v_avg', 'w2v_index',
                'd2v', and 'bow'.
            emb_dim: int, dimention of embedding; this is used
                all of methods except for 'bow'.
            revmoe_stopswords: bool, if True, remove stopwords
                downloaded by nltk
            remove_sords: bool, if True, remove charcters other than
                alphabets and numbers
            file_path: str, path to down load pretrained word2vec;
                this is used only for 'w2v', 's2v_avg', and
                'w2v_index'.

        Returns:
            np.array: embbeded vecotors or indices of words
        """
        if not remove_stopwords:
            remove_stopwords = self.remove_stopwords
        if not remove_special:
            remove_special = self.remove_special
        # Get text data in the format, list(str).
        sentences = self._preprocess_sentence(
                sentences, remove_stopwords, remove_special)
        # Convert into numeric data
        # Get bag of words.
        if self.embedding == "bow":
            _vectors = self.bow.transform(sentences).toarray()
        # Get word2vec and pad it with 0 vetors to have the fix length
        # and deal with unkonw words
        elif self.embedding == "w2v":
            _vectors = []
            for sentence in sentences:
                if self.emb_mode == 2:
                    _vector = [
                        np.concatenate((self.w2v[0][w], self.w2v[1][w]), axis=0)
                        for w in sentence if w in self.vocab.keys()]
                else:
                    _vector = [self.w2v[0][w] for w in sentence if w in self.vocab.keys()]
                _vectors.append(_vector)
            _vectors = pad_sequences(_vectors, maxlen=self.maxlen)
        # Get index to look up parameters for word2vec.
        elif self.embedding == "w2v_index":
            _vectors = []
            for sentence in sentences:
                _vector = []
                for w in sentence:
                    if w not in self.vocab.keys():
                        _vector.append(0)
                    else:
                        _vector.append(self.vocab[w])
                _vectors.append(_vector)
            # Unkown words and padding refer to 0 index, which returns
            # 0 vectors.
            _vectors = pad_sequences(_vectors, maxlen=self.maxlen)
        # Get word2vec averaged over each sentence
        elif self.embedding == "w2v_avg":
            _vectors = []
            # To use both of cbow and skip gram, we have 2*emb_dim
            # dimension
            zerovec = np.zeros(self.get_dim())
            for sentence in sentences:
                if self.emb_mode == 2:
                    _vector = [
                        np.concatenate((self.w2v[0][w], self.w2v[1][w]), axis=0)
                        for w in sentence if w in self.vocab.keys()]
                else:
                    _vector = [
                        self.w2v[0][w] for w in sentence if w in self.vocab.keys()]
                if not _vector:
                    _vector = [zerovec,]
                _vector = np.mean(_vector, axis=0)
                _vectors.append(_vector)
        # Get doc2vec, which combines two types doc2vec
        elif self.embedding == "d2v":
            _vectors = []
            for sentence in sentences:
                if self.emb_mode == 2:
                    _vector = np.concatenate(
                        (self.d2v[0].infer_vector(sentence.words),
                         self.d2v[1].infer_vector(sentence.words)),
                         axis=0)
                else:
                    _vector = self.d2v[0].infer_vector(sentence.words)
                _vectors.append(_vector)
        return np.array(_vectors)

    def get_params(self):
        weights0 = np.zeros((1, self.emb_dim))
        if self.embedding == "w2v":
            # add poarameters for index 0
            params = [
                np.concatenate((vec.syn0, weights0), axis=0) for vec in self.w2v]
        elif self.embedding == "d2v":
            params = [
                np.concatenate((vec.syn0, weights0), axis=0) for vec in self.d2v]
        else:
            raise ValueError("We do not have parameters for"
                             "embedding={}".format(self.embedding))
        return params
    
    def get_dim(self):
        if self.embedding == "w2v"\
                or self.embedding == "w2v_avg"\
                or self.embedding == "d2v":
            if self.emb_mode == 0 or self.emb_mode == 1:
                return self.emb_dim
            else:
                return 2 * self.emb_dim
        elif self.embedding == "w2v":
            return 0
        elif self.embedding == "bow":
            return None            

    def _preprocess_sentence(self, sentences,
                             remove_stopwords=None, remove_special=None):
        """Remove stopwords or special characters if necessary

        Args:
            sentences: list(str)
            revmoe_stopswords: bool, if True, remove stopwords
                downloaded by nltk
            remove_sords: bool, if True, remove charcters other than
                alphabets and numbers

        Returns:
            list(sentences)
        """
        _sentences = []
        if not remove_stopwords:
            remove_stopwords = self.remove_stopwords
        if not remove_special:
            remove_special = self.remove_special
        for i, sentence in enumerate(sentences):
            sentence = nltk.word_tokenize(sentence)
            sentence = " ".join(sentence)
            # Remove special characters and split a sentence into
            # a list of words
            if remove_special:
                _sentence = re.sub(r"[^a-z]", " ", sentence).split()
            else:
                _sentence = sentence.split()
            # Remove stopwords, e.g., he, she, you, ...
            if remove_stopwords:
                stops = set(stopwords.words("english"))
                _sentence = [w for w in _sentence if w not in stops]
            if self.embedding == "bow":
                _sentence = " ".join(_sentence)
            elif self.embedding == "w2v"\
                    or self.embedding == "w2v_avg"\
                    or self.embedding == "w2v_index":
                # Word2vec use a list of words as input
                pass
            elif self.embedding == "d2v":
                tag = "SENT_" + str(i)
                _sentence = TaggedDocument(words=_sentence, tags=[tag])
            else:
                raise ValueError("We do not have implementation for"
                                 "embedding={}".format(embedding))
            _sentences.append(_sentence)
        return _sentences

    def _initialize_processor(self, sentences, embedding, emb_mode, emb_dim,
                              remove_stopwords, remove_special, maxlen, file_path):
        """Initialize a processor and keep parameters as attirbutes.

        Args:
            sentences: list(str)
            embedding: str, name of methods of mebeddings; we
                currently have 'w2v', 'w2v_avg', 'w2v_index',
                'd2v', and 'bow'.
            emb_mode: int, mode of vector representation. If 2
                is chosen, emb_dim becomes double
            emb_dim: int, dimention of embedding; this is used
                all of methods except for 'bow'.
            revmoe_stopswords: bool, if True, remove stopwords
                downloaded by nltk
            remove_sords: bool, if True, remove charcters other than
                alphabets and numbers
            file_path: str, path to use pretrained word2vec;
                this is used only for 'w2v', 's2v_avg', and
                'w2v_index'.
        """
        # Initialize parameters and keep them as attributes.
        self.embedding = embedding
        self.emb_mode = emb_mode
        self.emb_dim = emb_dim
        self.file_path = file_path
        self.remove_stopwords = remove_stopwords
        self.remove_special = remove_special
        self.maxlen = maxlen
        if sentences is not None:
            lengthes = [len(x) for x in sentences]
            self.maxlen = int(np.mean(lengthes))
            sentences = self._preprocess_sentence(sentences,
                                              remove_stopwords,
                                              remove_special)
        if embedding == "bow":
            self.bow = CountVectorizer(analyzer="word", tokenizer=None,
                                       preprocessor=None, stop_words=None,
                                       max_features=5000)
            self.bow.fit(sentences)
        elif embedding == "w2v"\
                or embedding == "w2v_avg"\
                or embedding == "w2v_index":
            # mode=1: cbow, mode=1: skip-gram, and mode=2: both
            self.w2v = []
            if self.file_path is not None:
                vec = KeyedVectors.load_word2vec_format(self.file_path,
                                                        binary=True)
                self.w2v.append(vec)
                self.emb_mode = 0
            else:
                if self.emb_mode == 0 or self.emb_mode == 2:
                    cbow = gensim.models.word2vec.Word2Vec(
                    sentences, iter=6, size=emb_dim,
                    window=5, min_count=3, workers=4, max_vocab_size=50000, sg=0)
                    self.w2v.append(cbow)
                if self.emb_mode == 1 or self.emb_mode == 2:
                    skpg = gensim.models.word2vec.Word2Vec(
                        sentences, iter=6, size=emb_dim,
                        window=5, min_count=3, workers=4, max_vocab_size=50000, sg=1)
                    self.w2v.append(skpg)
            # Keep index 0 for unknown or padding.
            self.vocab = {}
            for word in iter(self.w2v[0].wv.vocab.keys()):
                self.vocab[word] = self.w2v[0].wv.vocab[word].index + 1
        elif embedding == "d2v":
            # mode=0: pvdbow, mode=1: pvdm, and mode=2: both
            self.d2v = []
            if self.emb_mode == 0 or self.emb_mode == 2:
                if file_path is None:
                    pvdbow = Doc2Vec(sentences, size=emb_dim, dm=0, alpha=1e-2,
                                 min_alpha=1e-3, sample=1e-5, 
                                window=15, min_count=3, workers=4,
                                max_vocab_size=50000, iter=6)
                else:
                    pvdbow = Doc2Vec.load(file_path)
                    self.emb_mode = 0
                self.d2v.append(pvdbow)
            if self.emb_mode == 1 or self.emb_mode == 2:
                if file_path is None:
                    pvdm = Doc2Vec(sentences, size=emb_dim, dm=1, alpha=1e-2,
                                min_alpha=1e-3, sample=1e-6, 
                                window=5, min_count=3, workers=4,
                                max_vocab_size=50000, iter=6)
                else:
                    pvdm = Doc2Vec.load(file_path)
                self.d2v.append(pvdm)

processor = SentenceProcessor(None, "w2v", maxlen=maxlen, vector_file_path="data/GoogleNews-vectors-negative300.bin")

Using TensorFlow backend.


CPU times: user 1min 43s, sys: 21.6 s, total: 2min 5s
Wall time: 2min 48s


In [5]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [6]:
X[1]

array(['What is the step by step guide to invest in share market?',
       'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?',
       'How can Internet speed be increased by hacking through DNS?', ...,
       "What's this coin?",
       'I am having little hairfall problem but I want to use hair styling product. Which one should I prefer out of gel, wax and clay?',
       'What is it like to have sex with your cousin?'], dtype=object)

In [7]:
default_sess = None
load_file_path = None
save_file_path = None
overwrite = True


nn_conf = {
        "learning_rate": 0.1,
        "learning_rate_minimum": 1e-5,
        "learning_rate_decay": 0.9,
        "learning_rate_decay_step": 10,
        "batch_size": 32,
        "model_dir": "./logs",
        "load_file_path": None,
        "save_file_path": None
}

In [21]:
import os
from os.path import join
import tensorflow as tf
from time import time 
from tqdm import tqdm
import numpy as np
from logging import getLogger
from copy import deepcopy
from tqdm import trange
import gc

is_processed, is_training = False, True

class BaseEstimator(object):
    """Abstract class for all estimators
    
    Each estimator is used with some mixin class, e.g., ClassifierMixin
    RegressorMixin. Do not use this abstract base class directly but 
    instead use one of the concrete estimators implemented.
    """
    def __init__(self, processor):
        """Initialize an estimators
        
        Args:
            model_params: dictionary, parameters of estimators, 
                e.g., output_dim and input_shape
            processor: helper class
        """
        self.processor = processor
        # Check if a model is optimized
        self.is_trained = False

    def load_params(self, filepath):
        """Loads parameters of an estimator from a file.
        
        Args:
            filepath: str, The path to the file.
        """
        raise NotImplementedError()

    def save_params(self, filepath, overwrite):
        """Saves parameters of an estimator as a file.
        
        Args:
            filepath: str, The path to where 
                the parameters should be saved.
            overwrite: bool, If `False` and `filepath` 
                already exists, raises an error.
        """
        raise NotImplementedError()
        
    def _calc_output(self, X, *args, **kwargs):
        """Return output of estimator
        
        Note:
            output of estimator is not necessary 
                prediction of target values.
            This value can be values necessary 
                for transformation, e.g. logit
        """
        raise NotImplementedError()


class NNTrainMixin(object):
    def train(self, train_X, train_y=None, valid_X=None, valid_y=None, num_epochs=100, 
              valid_freq=1, epoch_bar=True, batch_bar=False, log_freq=1, batch_log_freq=None,
              img_log_freq=None, plot_X=None, is_processed=False, 
              is_memory_intensive=False, overwrite=True, save_file_path=None, *args, **kwargs):
        """Train neural network model
        
        Args:
            train_X, train_y: list(array-like), this is preprocessed by 
                self.processor.batch_process
            valid_X, valid_y: array-like(optional), if they are feeded, they
                will be used as validation set
            is_processed: bool, if True, skip preprocessing
            args, kwargs: parameters for score function
        """
        train_X, train_y = self._preprocess(train_X, train_y)
        valid_X, valid_y = self._preprocess(valid_X, valid_y)
        # check if there is already the same name file
        self.save_params(self.save_file_path, overwrite)
        self.is_trained = True
        num_train = len(train_X[0])
        indices = np.arange(num_train)
        try:
            global_batch_step = 0
            if epoch_bar:
                epoch_iter = trange(num_epochs)
            else:
                epoch_iter  = range(num_epochs)
            for i in epoch_iter:
                np.random.shuffle(indices)
                self.sess.run(self.update_step_op);
                epoch_loss = []
                epoch_score = []
                batch_loss_list = []
                batch_score_list = []
                batch_step = 0
                total_batch = num_train//self.batch_size + 1
                if batch_bar:
                    batch_iter = trange(total_batch)
                else:
                    batch_iter = range(total_batch)
                for batch_i in batch_iter:
                    batch_step += 1
                    global_batch_step += 1
                    batch_idx = self._get_batch_idx(batch_i, indices)
                    batch_X = [X[batch_idx] for X in train_X]
                    if not is_processed:
                        batch_X = [self.processor.batch_process(X_i) for X_i in batch_X]
                    if train_y is not None:
                        batch_y = train_y[batch_idx]
                    else:
                        batch_y = None
                    batch_loss = self._optimize(batch_X, batch_y, num_data=num_train)
                    if batch_y is not None:
                        batch_score = self.score(batch_X, batch_y, is_training=False, is_processed=True)
                        batch_score_list.append(batch_score)
                        epoch_score.append(batch_score)
                    if batch_log_freq is not None: 
                        batch_loss_list.append(batch_loss)
                        if batch_step % batch_log_freq == 0:
                            tag_dict = {"batch_loss": np.mean(batch_loss_list)}
                            if batch_score_list:
                                tag_dict["batch_score"] = np.mean(batch_score_list)
                            self._inject_summary(tag_dict, global_batch_step)
                            batch_loss_list = []
                            batch_score_list = []
                    epoch_loss.append(np.mean(batch_loss))
                    if is_memory_intensive:
                        del batch_X
                        del batch_y
                        gc.collect()
                step = self.global_step.eval(session=self.sess)
                if step % log_freq == 0:
                    lr_val = self.learning_rate_op.eval(session=self.sess)
                    tag_dict = {'loss': np.mean(epoch_loss), "learning_rate":lr_val}
                    if epoch_score:
                        tag_dict["score"] = np.mean(epoch_score)
                    self._inject_summary(tag_dict)
                # self._epoch_func(X=train_X, y=train_y, is_processed=is_processed)
                accuracies = []
                if valid_y is not None:
                    num_valid = len(valid_y)
                    # check accuracy every print_freq epoch
                    if step % valid_freq == 0:
                        accuracies = []
                        logits_list = []
                        for batch_i in range(num_valid//self.batch_size + 1):
                            batch_idx = self._get_batch_idx(batch_i)
                            batch_X = [X[batch_idx] for X in valid_X]
                            batch_y = valid_y[batch_idx]
                            _score = self.score(batch_X, batch_y, is_training=False,
                                                is_processed=True, *args, **kwargs)
                            accuracies.append(_score)
                        accuracy = np.mean(accuracies)
                        print("accuracy: ", accuracy)
        except KeyboardInterrupt:
            print("Save model parameters before finishing training...")
        finally:
            self.save_params(save_file_path, overwrite=True)
        print("finished training")
        
    def _optimize(self, batch_X, batch_y, *args, **kwargs):
        feed_dict={
                self.input: batch_X[0],
                self.training: True}
        if batch_y is not None:
            feed_dict[self.target] = batch_y
        _, loss = self.sess.run([self.train_step, self.loss], feed_dict=feed_dict)
        return loss
    
    def _epoch_func(self, X, y, *args, **kwargs):
        pass
    
    def _gen_plot(self, X):
        raise NotImplementedError()
    
    def _inject_summary(self, tag_dict, step=None):
        if step is None:
            step = self.global_step.eval(session=self.sess)
        summary_str_lists = self.sess.run([self.summary_ops[tag] for tag in tag_dict.keys()], {
          self.summary_placeholders[tag]: value for tag, value in tag_dict.items()
        })
        for summary_str in summary_str_lists:
            self.writer.add_summary(summary_str, step)
            
    def _get_tag_dict(self, epoch_loss, *args, **kwargs):
        lr_val = self.learning_rate_op.eval(session=self.sess)
        tag_dict = {'loss': np.mean(epoch_loss), "learning_rate":lr_val}
        return tag_dict
    
    def _preprocess(self, X, y):
        return [X], y
    
    def _get_batch_idx(self, idx, indices):
        return indices[idx*self.batch_size: (idx+1)*self.batch_size]        

class SaveLoadMixin(object):

    def load_params(self, file_path):
        """Loads parameters of an estimator from a file.
        
        Args:
            file_path: str, The path to the file.
        """
        self.saver.restore(self.sess, file_path)
        print("Model restored.")

    def save_params(self, file_path=None, overwrite=True):
        """Saves parameters of an estimator as a file.
        
        Args:
            file_path: str, The path to where the parameters should be saved.
            overwrite: bool, If `False` and `file_path` already exists, raises an error.
        """
        if file_path is None:
            if not os.path.isdir("params"):
                os.mkdir("params")
            file_path = "params/model.ckpt"
        if not overwrite:
            _path = ".".join([file_path, "meta"])
            if os.path.isfile(_path):
                raise NameError("%s already exists." % file_path)
        save_path = self.saver.save(self.sess, file_path)
        print("Model saved in file: %s" % save_path)


class BaseNN(SaveLoadMixin, NNTrainMixin, BaseEstimator):
    """Abstract class for neural networks.
    
    Do not use this class directly. You should use this class with other mixin 
    class, e.g., NNClassifier and  NNRegressor.     
    """
    def __init__(self, input_dim, output_dim, model, conf=None,
                 sess=None, default_conf=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.conf = conf
        if sess is None:
            sess = tf.Session()
        self.sess = sess
        # configuration for not specified features
        if default_conf is None:
            default_conf = nn_conf
        self.default_conf = default_conf
        self.conf = self._set_conf(conf)
        self.model = model(output_dim, self.conf["model"], sess=self.sess, *args, **kwargs)
        self.input_dim = input_dim
        self.output_dim = output_dim
        # configure for learning schedule
        self.learning_rate = self.conf["learning_rate"]
        self.learning_rate_decay_step = self.conf["learning_rate_decay_step"]
        self.learning_rate_decay = self.conf["learning_rate_decay"]
        self.learning_rate_minimum = self.conf["learning_rate_minimum"]
        self.batch_size = self.conf["batch_size"]
        # directory for tensorboard
        self.model_dir = self.conf["model_dir"]
        print("Building tensorflow graph...")
        st = time()
        with self.sess.as_default():
            # build tensorflow graph
            self.global_step = tf.Variable(0, name="epoch", trainable=False)
            self.update_step_op = self._get_update_step_op()
            self._build_graph()
            self.saver = tf.train.Saver()
        print("Finished building tensorflow graph, spent time:", time() - st)
        self.save_file_path = self.conf["save_file_path"]
        load_file_path = self.conf["load_file_path"]
        if load_file_path is not None:
            self.load_params(load_file_path)
            self.is_trained = True

    def _build_graph(self):
        raise NotImplementedError()
        
    def _get_learning_rate(self):
        learning_rate_op = tf.maximum(self.learning_rate_minimum,
              tf.train.exponential_decay(
              self.learning_rate,
              self.global_step,
              self.learning_rate_decay_step,
              self.learning_rate_decay,
              staircase=False))
        return learning_rate_op

    def _calc_output(self, X, is_training=is_training, is_processed=is_processed):
        """Return model output
        
        Returns:
            array-like
        """
        # Check the shape
        if isinstance(self.input_dim, int):
            dim_len = 1 + 1
        else:
            dim_len = len(self.input_dim) + 1
        if len(np.array(X).shape) > dim_len:
            X = X[0]
        if not is_processed:
            X = self.processor.batch_process(X)
        num_data = len(X)
        output_list = []
        for batch_i in range(num_data // self.batch_size + 1):
            batch_X = X[self.batch_size * batch_i : self.batch_size * (batch_i+1)]
            feed_dict = {self.training: is_training, self.input: batch_X}
            output_list.extend(self.sess.run(self.output, feed_dict=feed_dict))
        return np.array(output_list)
    
    def _set_conf(self, conf):
        conf = deepcopy(conf)
        for key in self.default_conf.keys():
            if key not in conf:
                conf[key] = self.default_conf[key]
        return conf
    
    def _get_update_step_op(self):
        return tf.assign(self.global_step, self.global_step + 1)
    
    def _build_summaries(self):
        self.writer = tf.summary.FileWriter(self.model_dir, self.sess.graph)
        self.summary_placeholders = {}
        self.summary_ops = {}
        scalar_summary_tags = ['loss', 'score', 'learning_rate', 'batch_loss', 'batch_score']
        for tag in scalar_summary_tags:
            self.summary_placeholders[tag] = tf.placeholder('float32', None, name=tag.replace(' ', '_'))
            self.summary_ops[tag] =\
                tf.summary.scalar("%s" % tag, self.summary_placeholders[tag])
                
class MultiNN(BaseNN):
    def __init__(self, input_dim, output_dim, input_model, output_model, conf=None,
                 sess=None, default_conf=None, *args, **kwargs):
        super().__init__(input_dim, output_dim, output_model, conf,
                 sess, default_conf, *args, **kwargs)
        
    def _calc_output(self, X, is_training=is_training, is_processed=is_processed):
        """Return model output
        
        Returns:
            list(array-like)
        """
        if not self.is_trained:
            raise NotTrainedError("Train model by 'train' beforehand")
        if not is_processed:
            X = [self.processor.batch_process(_X) for _X in X]
        num_data = len(X[0])
        output_list = []
        for batch_i in range(num_data // self.batch_size + 1):
            feed_dict = {self.training: is_training}
            for _input, _X in zip(self.input, X):
                batch_X = _X[self.batch_size * batch_i : self.batch_size * (batch_i+1)]
                feed_dict[_input] = batch_X
            output_list.extend(self.sess.run(self.output, feed_dict=feed_dict))
        return np.array(output_list)
    
    def _optimize(self, batch_X, batch_y, *args, **kwargs):
        feed_dict={self.training: True}
        if batch_y is not None:
            feed_dict[self.target] = batch_y
        for input_, X in zip(self.input, batch_X):
            feed_dict[input_] = X
        _, loss = self.sess.run([self.train_step, self.loss], feed_dict=feed_dict)
        return loss
    
    def _preprocess(self, X, y):
        return X, y

In [27]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected, conv2d
from tensorflow.contrib.layers import conv2d_transpose, flatten
import numpy as np
from copy import deepcopy
from tensorflow.contrib.rnn import MultiRNNCell
from tensorflow.contrib import rnn
from tensorflow.contrib.rnn import *

class BaseModel(object):
    def __init__(self, model_params, scope_name, *args, **kwargs):
        self.model_params = model_params
        self.scope_name = scope_name
        self.reuse = False
        
    def __call__(self, x, training=None):
        raise NotImplementedError()

    def get_input(self):
        return self.input
    
    def get_output(self):
        return self.output

def get_shape(input_shape, is_batch=True, is_sequence=False, maxlen=None):
    """Get shape of batch input for model
    
    Args:
        input_shape: int or tuple, shape of input 
        is_batch: bool, if True, we add another batch
            dimension, None
    
    Returns:
        tuple: if is_batch is False, shape=(None, *input_shape)
    """
    if isinstance(input_shape, int):
        input_shape = (input_shape,)
    elif isinstance(input_shape, list):
        input_shape = tuple(input_shape)
    if is_sequence:
        input_shape = (maxlen,) + input_shape
    if is_batch:
        input_shape = (None,) + input_shape
    return input_shape


def get_length(sequence):
    """Get length of each instance in batch
    
    Args:
        sequence: tensor, shape = (batch_size, length)
            or shape = (batch_size, length) + shape
            
    Returns:
        tensor: shape = (None,), length of each instance
    """
    shape = sequence.get_shape().as_list()
    if len(shape) < 2:
        used = tf.sign(tf.abs(sequence))
    else:
        reduction_indices = list(range(2, len(shape)))
        used = tf.sign(tf.reduce_max(tf.abs(sequence),
                       reduction_indices=reduction_indices))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length


        
class FeedForward(BaseModel):
    def __init__(self, model_params, scope_name, *args, **kwargs):
        super().__init__(model_params, scope_name, *args, **kwargs)
    
    def __call__(self, x, training=True):
        with tf.variable_scope(self.scope_name, reuse=self.reuse):
            for i, params in enumerate(self.model_params):
                with tf.variable_scope('layer_' + str(i)):
                    if "is_flatten" in params and params["is_flatten"]:
                        x = flatten(x)
                    if "drop_rate" in params:
                        x = tf.layers.dropout(x, rate=params["drop_rate"], training=training)
                    # demtermine which layer to use
                    if params["name"] == "dense":
                        x = fully_connected(x, params["num_hidden"], activation_fn=None)
                    elif params["name"] == "conv2d":
                        x =  conv2d(x, params["num_filter"], params["kernel_size"],
                                          params["stride"], params["padding"], activation_fn=None)
                    elif params["name"] == "deconv2d":
                        x =  conv2d_transpose(x, params["num_filter"], params["kernel_size"],
                                          params["stride"], params["padding"], activation_fn=None)
                    elif params["name"] == "reshape":
                        x = tf.reshape(x, (-1,) + params["reshape_size"])
                    elif params["name"] == "pooling":
                        del params["name"]
                        x = tf.nn.pool(x, **params)
                    else:
                        raise NotImplementedError("No implementation for 'name'={}".format(params["name"]))         
                    if "is_batch" in params and params["is_batch"]:
                        x = tf.layers.batch_normalization(x, training=training, momentum=0.9)
                    if "activation" in params:
                        x = params["activation"](x)
            if self.reuse is False:
                self.global_scope_name = tf.get_variable_scope().name
                self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=self.global_scope_name)
        self.reuse = True
        return x


lnlstm_cell_params = {"forget_bias": 1.0, "activation": tf.nn.tanh,
                      "layer_norm": True, "norm_gain": 1.0, "norm_shift": 0.0,
                      "drop_rate": 0.0, "dropout_seed": None, 
                      "reuse": None}

maxlen=None

def get_cell(params):
    _params = deepcopy(params)
    if _params["name"] == "lnlstm":
        del _params["name"]
        return LNLSTMCell(**_params)
    elif _params["name"] == "lstm":
        del _params["name"]
        return LSTMCell(**_params)
    elif _params["name"] == "gru":
        del _params["name"]
        return GRUCell(**_params)
    else:
        raise NotImplementedError()
        

class LNLSTMCell(rnn.RNNCell):
    def __init__(self, num_units, forget_bias=lnlstm_cell_params["forget_bias"],
                 activation=lnlstm_cell_params["activation"],
                 layer_norm=lnlstm_cell_params["layer_norm"], 
                 norm_gain=lnlstm_cell_params["norm_gain"], 
                 norm_shift=lnlstm_cell_params["norm_shift"],
                 drop_rate=lnlstm_cell_params["drop_rate"], 
                 dropout_seed=lnlstm_cell_params["dropout_seed"], 
                 reuse=lnlstm_cell_params["reuse"]):
        self._num_units = num_units
        self._activation = activation
        self._forget_bias = forget_bias
        self._layer_norm = layer_norm
        self._keep_prob = 1 - drop_rate
        self._seed = dropout_seed
        self._g = norm_gain
        self._b = norm_shift
        self._reuse = reuse
        
    @property
    def state_size(self):
        return rnn.LSTMStateTuple(self._num_units, self._num_units)
    
    @property
    def output_size(self):
        return self._num_units
    
    def _norm(self, input_, scope):
        # Keep list format
        shape = input_.get_shape()[-1:]
        gamma_init = tf.constant_initializer(self._g)
        beta_init = tf.constant_initializer(self._b)
        with tf.variable_scope(scope):
            tf.get_variable("gamma", shape=shape,
                            initializer=gamma_init, dtype=tf.float32)
            tf.get_variable("beta", shape=shape,
                            initializer=beta_init, dtype=tf.float32)
        normalized = tf.contrib.layers.layer_norm(input_, reuse=True,
                                                  scope=scope)
        return normalized
    
    def _linear(self, x):
        out_size = 4 * self._num_units
        proj_size = x.get_shape()[-1]
        weights = tf.get_variable("kernel", [proj_size, out_size],
                                  dtype=tf.float32)
        out = tf.matmul(x, weights)
        if not self._layer_norm:
            bias = tf.get_variable("bias", [out_size])
            out = tf.nn.bias_add(out, bias)
        return out
    
    def __call__(self, inputs, state, scope=None):
        c, h = state
        xh = tf.concat([inputs, h], 1)
        out_size = 4 * self._num_units
        xh = self._linear(xh)
        
        i, j, f, o = tf.split(value=xh, num_or_size_splits=4, axis=1)
        if self._layer_norm:
            i = self._norm(i, "input")
            j = self._norm(j, "transform")
            f = self._norm(f, "forget")
            o = self._norm(o, "output")
        
        g = self._activation(j)
        if (not isinstance(self._keep_prob, float)) or self._keep_prob < 1:
            g = tf.nn.dropout(g, self._keep_prob, seed=self._seed)
            
        new_c = (c * tf.sigmoid(f + self._forget_bias) + tf.sigmoid(i) * g)
        
        if self._layer_norm:
            new_c = self._norm(new_c, "state")
        new_h = self._activation(new_c) * tf.sigmoid(o)
        new_state = rnn.LSTMStateTuple(new_c, new_h)
        return new_h, new_state

class BaseRNN(BaseModel):
    def __init__(self, model_params, scope_name, full_output=False, *args, **kwargs):
        super().__init__(model_params, scope_name, *args, **kwargs)
        self.full_output = full_output
    
    def __call__(self, x, training=True):
        with tf.variable_scope(self.scope_name, reuse=self.reuse):
            if isinstance(self.model_params, list) or isinstance(self.model_params, tuple):
                cells = []
                for i, params in enumerate(self.model_params):
                    with tf.variable_scope('cell_' + str(i)):
                        cells.append(get_cell(params))
                cell = MultiRNNCell(cells)
            else:
                cell = get_cell(self.model_params)
            outputs, state = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32, sequence_length=get_length(x))
            if self.reuse is False:
                self.global_scope_name = tf.get_variable_scope().name
                self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=self.global_scope_name)
        self.reuse = True
        if self.full_output:
            return outputs
        else:
            return state.h
        
class RNNModel(BaseRNN):
    def __init__(self, output_dim=None, model_params=None, scope_name=None, *args, **kwargs):
        if model_params is None:
            model_params = rnn_conf["model"]
        if scope_name is None:
            scope_name = "rnn"
        self.rnn_model = BaseRNN(model_params, scope_name, *args, **kwargs)
        if output_dim is not None:
            self.ff_model = FeedForward([{"name": "dense", "num_hidden": output_dim},], scope_name)
        else:
            self.ff_model = None
        super().__init__(model_params, scope_name, *args, **kwargs)   
        
    def __call__(self, x, training=True):
        x = self.rnn_model(x, training)
        if self.ff_model is not None:
            x = self.ff_model(x, training)
        return x
    
class MLPModel(FeedForward):
    def __init__(self, output_dim=None, model_params=None, scope_name=None, *args, **kwargs):
        if model_params is None:
            model_params = mlp_conf["model"]
        if scope_name is None:
            scope_name = "mlp"
        if output_dim  is not None:
            model_params.append({"name": "dense", "num_hidden": output_dim})
        super().__init__(model_params, scope_name, *args, **kwargs)

In [28]:
def accuracy_score(y, y_pred, *args, **kwargs):
    y = np.squeeze(y)
    y_pred = np.squeeze(y_pred)
    return np.mean(np.equal(y, y_pred))

def sigmoid(x):
    return 1. / (1. + np.exp(-x))


def softmax(x):
    shape = x.shape
    z = np.sum(np.exp(x), axis=1, keepdims=True)
    z = np.tile(z, (1, shape[1]))
    return np.exp(x) / z


class ClassifierMixin(object):
    """Mixin class for all classifiers
    
    Do not use this mixin class directly.
    """
    _estimator_type = "classifier"

    def score(self, X, y, score_func=accuracy_score, *args, **kwargs):
        """Returns the score on the input data and labels.
        
        If score_func is not specified, we use accuracy function.
        
        Args:
            X : array-like
            y : array-like, shape = (n_samples) True labels for X.
            score_func: optional, metric function
            is_training: bool, output mode
                this will be used when using certain regularizers like
                dropout
            args, kwargs: additional parameters for self.predict
        
        Returns:
            float
        """
        return score_func(y, self.predict(X, *args, **kwargs))
    
    def predict_conf(self, X, *args, **kwargs):
        """Return confidence of predictions
        
        Binary or multi classification will be specified by 
        self.output_dim. If you want to use output of an estimator
        as confidence directly, you should set 'self.is_logit=False'.
        
        Args:
            X : array-like
            is_training: bool, output mode
                this will be used when using certain regularizers like 
                dropout
            args, kwargs: additional parameters for self._calc_output
        
        Returns:
            array_like, shape = (n_samples, self.output_dim)
        """
        if self.is_logit:
            logit = self._calc_output(X, *args, **kwargs)
            if self.output_dim == 1:
                prediction = sigmoid(logit)
            else:
                prediction = softmax(logit)
        else:
            prediction = self._calc_output(X, *args, **kwargs)
        # Keep shape = (n_samples, self.output_dim)
        if len(prediction.shape) == 1:
            prediction = prediction[:, np.newaxis]
        return prediction
    
    def predict(self, X, threshold=0.5, *args, **kwargs):
        """Return predictions of an estimator
        
        Binary or multi classification will be specified by
        self.output_dim. If labels are binary, we assigne 0 to negative 
        and 1 to positive isntances. If you want to use output of
        an estimator as confidence directly, you should set 
        'self.is_logit=False'.
        
        Args:
            X : array-like
            is_training: bool, output mode
                this will be used when using certain regularizers like 
                dropout
            threshold: float, this will be used only for binary 
                classification
            args, kwargs: additional parameters for self._calc_output
        
        Returns:
            array_like, shape = (n_samples, self.output_dim)
        """
        conf = self.predict_conf(X, *args, **kwargs)
        output_shape = conf.shape
        if output_shape[-1] == 1:
            prediction = np.array([1 if x[0] > threshold else 0 for x in conf])
        else:
            prediction = np.argmax(conf, axis=1)
        return prediction

    
nn_is_logit = True

class NNClassifier(BaseNN, ClassifierMixin):
    """Classifier based on neural network"""
    
    def __init__(self, input_dim, output_dim, model, conf,
                 is_sequence, is_logit=nn_is_logit, *args, **kwargs):
        """Initialize classifier with nerual network model
        
        Args:
            model: class, neural network class
            is_logit: bool(optional), if False, output of an estimator
                as prediction directly
            args, kwargs: parameters for parents class
        """
        self.is_logit = is_logit
        self.is_sequence = is_sequence
        super().__init__(input_dim, output_dim, model=model,
                         conf=conf, *args, **kwargs)
        
    def _build_graph(self):
        """Build tensorflow graph
        
        Note:
            You build graphs for output and input, which will be used 
            for training and prediction.
        """
        _input_dim = get_shape(self.input_dim, is_sequence=self.is_sequence)
        self.epoch = tf.Variable(0, name="epoch", trainable=False)
        self.input = tf.placeholder(tf.float32, shape=_input_dim)
        self.target = tf.placeholder(tf.int32, shape=(None,), name="target")
        self.training = tf.placeholder(tf.bool, (), name="training")
        self.output = self.model(self.input, self.training)
        
        # build optimizer
        if self.is_logit:
            if self.output_dim==1:
                _output = tf.squeeze(self.output)
                _loss =\
                    tf.nn.sigmoid_cross_entropy_with_logits(
                        labels=tf.cast(self.target, tf.float32),
                        logits=_output)
            else:
                _target = tf.one_hot(self.target, self.output_dim)
                _loss =\
                    tf.nn.softmax_cross_entropy_with_logits(
                        labels=tf.cast(_target, tf.float32),
                        logits=self.output)
            self.loss = tf.reduce_mean(_loss)
        else:
            raise NotImplementedError("We have not implemeted non logit output model")
        self.learning_rate_op = self._get_learning_rate()
        self.train_step =\
            tf.train.AdamOptimizer(self.learning_rate_op).minimize(self.loss)
        # Build tensorboad graph
        with tf.name_scope("summary"):
            self._build_summaries()
        
        # initialize graph
        self.sess.run(tf.global_variables_initializer())
        
class MultiNNClassifier(MultiNN, ClassifierMixin):
    """Classifier based on neural network"""
    
    def __init__(self, input_dim, output_dim, input_model, output_model, conf,
                 is_sequence, is_logit=nn_is_logit, *args, **kwargs):
        """Initialize classifier with nerual network model
        
        Args:
            model: class, neural network class
            is_logit: bool(optional), if False, output of an estimator
                as prediction directly
            args, kwargs: parameters for parents class
        """
        self.is_logit = is_logit
        self.is_sequence = is_sequence
        self.input_model = []
        for i, model in enumerate(input_model):
            self.input_model.append(model(None, conf["input_model"][i], "input_model_%d" % i))
        self.output_model = output_model(output_dim, conf["model"], "output_model")
        super().__init__(input_dim, output_dim, input_model=input_model,
                         output_model=output_model, conf=conf, *args, **kwargs)
        
        
    def _build_graph(self):
        """Build tensorflow graph
        
        Note:
            You build graphs for output and input, which will be used 
            for training and prediction.
        """
        self.epoch = tf.Variable(0, name="epoch", trainable=False)
        self.input = []
        for _input_dim in self.input_dim:
            _input_dim = get_shape(_input_dim, is_sequence=self.is_sequence)
            self.input.append(tf.placeholder(tf.float32, shape=_input_dim))
        self.target = tf.placeholder(tf.int32, shape=(None,), name="target")
        self.training = tf.placeholder(tf.bool, (), name="training")
        outputs = []
        for _input, model in zip(self.input, self.input_model):
            outputs.append(model(_input, self.training))
        output_input = tf.concat(outputs, axis=-1)
        self.output = self.output_model(output_input)
        
        # build optimizer
        if self.is_logit:
            if self.output_dim==1:
                _output = tf.squeeze(self.output)
                _loss =\
                    tf.nn.sigmoid_cross_entropy_with_logits(
                        labels=tf.cast(self.target, tf.float32),
                        logits=_output)
            else:
                _target = tf.one_hot(self.target, self.output_dim)
                _loss =\
                    tf.nn.softmax_cross_entropy_with_logits(
                        labels=tf.cast(_target, tf.float32),
                        logits=self.output)
            self.loss = tf.reduce_mean(_loss)
        else:
            raise NotImplementedError("We have not implemeted non logit output model")
        self.learning_rate_op = self._get_learning_rate()
        self.train_step =\
            tf.train.AdamOptimizer(self.learning_rate_op).minimize(self.loss)
        # Build tensorboad graph
        with tf.name_scope("summary"):
            self._build_summaries()
        
        # initialize graph
        self.sess.run(tf.global_variables_initializer())

In [29]:
mlp_conf = {
    "model": [
        {"name": "dense", "num_hidden": 100, "is_batch":True, 'activation': tf.tanh},
        {"name": "dense", "num_hidden": 100, "drop_rate": 0.5, "is_batch":True, 'activation': tf.tanh}, 
        {"name": "dense", "num_hidden": 10, "drop_rate": 0.5, "is_batch":True, 'activation': tf.tanh}
    ]
}


rnn_conf = {"model":{"name":"lnlstm", "num_units":100}}

lnlstm_cell_params = {"forget_bias": 1.0, "activation": tf.nn.tanh,
                      "layer_norm": True, "norm_gain": 1.0, "norm_shift": 0.0,
                      "drop_rate": 0.0, "dropout_seed": None, 
                      "reuse": None}

maxlen=None

In [30]:
%%time


conf = {"model":[{"name": "dense", "num_hidden": 10, "is_batch": True, "activation": tf.nn.relu}],
        "input_model":[{"name":"lnlstm", "num_units":256, "activation":tf.nn.tanh,
                         "layer_norm":True, "drop_rate":0.0},
                       {"name":"lnlstm", "num_units":256, "activation":tf.nn.tanh,
                         "layer_norm":True, "drop_rate":0.0},],
        "learning_rate": 1e-3,
        "learning_rate_decay_step": 10,
        "learning_rate_decay": 0.9,
        "learning_rate_minimum": 1e-4,
        "batch_size":128,
        "load_file_path": "./lstm_params/model.ckpt"
       }


tf.reset_default_graph()
input_model = [RNNModel, RNNModel]
output_model = MLPModel
model = MultiNNClassifier([300, 300], 1, input_model, output_model, conf, processor=processor, is_sequence=True)

Building tensorflow graph...
Finished building tensorflow graph, spent time: 24.54978585243225
INFO:tensorflow:Restoring parameters from ./lstm_params/model.ckpt
Model restored.
CPU times: user 20.7 s, sys: 3.47 s, total: 24.1 s
Wall time: 25.6 s


In [43]:
model.predict_conf([[X[0][0],], [X[1][0]]])

array([[ 0.65194649]], dtype=float32)

In [42]:
X[0][0]

'What is the step by step guide to invest in share market in india?'

In [44]:
X[0][0]

'What is the step by step guide to invest in share market in india?'

In [45]:
X[1][0]


'What is the step by step guide to invest in share market?'